In [42]:
import spectHR as cs
import pyhrv
from ipywidgets import Tab, Output
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use('ggplot')
cs.handler.show_logs()

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [44]:
DataSet = cs.SpectHRDataset("SUB_005.xdf", use_webdav=True, reset = False)
DataSet = cs.borderData(DataSet)
DataSet = cs.filterECGData(DataSet, {"filterType": "highpass", "cutoff": .50})
if not hasattr(DataSet, 'RTops'):
    DataSet = cs.calcPeaks(DataSet)

In [46]:
%matplotlib widget
GUI = cs.prepPlot(DataSet)

preProcessing =  Output()
with preProcessing:
    display(GUI)

poincarePlot = Output()
psdPlot = Output()
descriptives = Output()
Gantt = Output()

import pandas as pd

App = Tab(children=[preProcessing, poincarePlot, descriptives, psdPlot, Gantt])

App.set_title(0, "PreProcessing")
App.set_title(1, "Poincare")
App.set_title(2, "Descriptives")
App.set_title(3, "PSD")
App.set_title(4, "Epochs")

DataSet.psd_Values = pd.Series()
DataSet.descriptives_Values = pd.Series()

# This function is called everytime a tab is selected in the App. The value of change['new'] holds the index to the selected tab.
def on_tab_change(change):
    if change['name'] == 'selected_index':        
        if change['new'] == 1:  # Poincare selected
            with poincarePlot:
                poincarePlot.clear_output()
                display(cs.poincare(DataSet))
            
        if change['new'] == 2: # selected the descriptives tab
            with descriptives:
                descriptives.clear_output()
                global descriptives_Values
                DataSet.descriptives_Values = cs.explode(DataSet)\
                        .groupby('epoch')['ibi']\
                        .agg([\
                            ('N', len),\
                            ('mean', 'mean'),\
                            ('std', 'std'),\
                            ('min', 'min'),\
                            ('max', 'max'),\
                            ('rmssd', lambda x: pyhrv.time_domain.rmssd(x)[0]), \
                            ('sdnn', lambda x: pyhrv.time_domain.sdnn(x)[0]),\
                            ('sdsd', cs.Tools.Params.sdsd),\
                            ('sd1', cs.Tools.Params.sd1),\
                            ('sd2', cs.Tools.Params.sd2),\
                            ('sd_ratio', cs.Tools.Params.sd_ratio),\
                            ('ellipse_area', cs.ellipse_area)\
                        ])
                display(DataSet.descriptives_Values)
                
        if change['new'] == 3:  # psd selected
            with psdPlot:
                psdPlot.clear_output()    
                Data = cs.explode(DataSet)
                DataSet.psd_Values = Data.groupby('epoch')[Data.columns.tolist()].apply(cs.welch_psd)
                display(DataSet.psd_Values)

        if change['new'] == 4:  # gantt selected 
            with Gantt:
                Gantt.clear_output()
                display(cs.gantt(DataSet, labels=False))
                
    DataSet.save()
                
# Attach observer to detect tab switch
App.observe(on_tab_change, names='selected_index')

display(App)

In [50]:
df = pd.DataFrame(list(DataSet.psd_Values.dropna()))
df['epoch'] = DataSet.psd_Values.dropna().index

In [52]:
pd.set_option('display.precision', 12)
df

,VLF Power,LF Power,HF Power,LF/HF Ratio,epoch
0,0.000418066882,0.000715733475,0.000549605374,1.302267970655,congruent
1,0.001404592979,0.001215593913,0.000779789127,1.558875175503,incongruent
2,0.000095451505,0.000147877932,0.000294239082,0.502577464967,names
3,0.000171365691,0.000611848688,0.000361344138,1.693257544732,words


In [54]:
pd.set_option('display.precision', 8)
pd.merge(DataSet.descriptives_Values, df, on = 'epoch', how='outer')

,epoch,N,mean,std,min,max,rmssd,sdnn,sdsd,sd1,sd2,sd_ratio,ellipse_area,VLF Power,LF Power,HF Power,LF/HF Ratio
0,congruent,641,0.80440196,0.05691557,0.61355615,0.97339411,41.73476518,56.91338610,25.38285175,29.51412308,74.60880761,0.39558497,6917.82953153,0.00041807,0.00071573,0.00054961,1.30226797
1,incongruent,574,0.78721678,0.08429569,0.59061920,0.99802798,48.36582527,84.29319781,31.81420122,34.18588478,114.19829165,0.29935548,12264.68234282,0.00140459,0.00121559,0.00077979,1.55887518
2,names,182,0.81933993,0.03299007,0.75055472,0.90920957,34.27126109,32.98280921,20.45985454,24.24049649,39.78469586,0.60929199,3029.75440634,0.00009545,0.00014788,0.00029424,0.50257746
3,words,173,0.78849094,0.04726513,0.64568929,0.88299081,34.32115762,47.24250859,20.37462616,24.26481240,61.83255652,0.39242777,4713.50545262,0.00017137,0.00061185,0.00036134,1.69325754
